In [1]:
from read_write import read
from stats import percentage, absolute
from viz import heatmap, bar
from utils import time, shape, string_op, probability
import numpy as np
from datetime import datetime
import dateutil
import pandas as pd

In [2]:
#symbol = "LEAF.TO"
#symbol = "APH.TO"
symbol = "WEED.TO"

end_date = datetime.today() #today
start_date = end_date + dateutil.relativedelta.relativedelta(months=-3) #month ago

date, num_data = read.get_data_from_web(symbol, start_date, end_date)

sh = np.shape(date)

days = []
for i in range (sh[0]):
#    day = time.get_day_number_from_date_string(date[i][0])
    day = time.get_day_number_from_date_string(date[i].strftime("%Y-%m-%d"))
    days.append(day)

#nd = read.get_numeric_data_from_all_data(data)
pc = percentage.get_percentage_change_from_numeric_data(num_data)
perc = pc * 100

range_pc = percentage.get_high_low_percentage_change_from_numeric_data(num_data)
range_perc = range_pc * 100

mod = 5

shaped_perc = shape.reshape_data(perc, days, mod)
shaped_range = shape.reshape_data(range_perc, days, mod)

title = symbol + ": " + start_date.strftime("%Y-%m-%d")+ " to " + end_date.strftime("%Y-%m-%d")

heatmap.show(shaped_perc, title, mod)


In [5]:
change = absolute.get_absolute_change_from_numeric_data(num_data)

#print change

sh = np.shape(change)

POSITIVE_TYPE = 1
NEGATIVE_TYPE = -1
current_type = 0 

#consec_pos_day_gain = []
#consec_neg_day_loss = []
consec_day = []

pos_start = 0
pos_end = 0

neg_start = 0
neg_end = 0

count = 0


for i in range (sh[0]):
    #if positive day
    if (change[i][0] > 0):
        if current_type == POSITIVE_TYPE:
            count = count + 1
            continue
        elif current_type == NEGATIVE_TYPE:
            neg_end = read.get_closing_price_from_numeric_data(num_data[i-1:i])[0][0]
            record = np.array([neg_start, neg_end, count])
            consec_day.append(record)
 
            pos_start = read.get_opening_price_from_numeric_data(num_data[i:i+1])[0][0]
            count = 1
        else:
            pos_start = read.get_opening_price_from_numeric_data(num_data[i:i+1])[0][0]
            count = 1
        current_type = POSITIVE_TYPE
    else:
        if current_type == NEGATIVE_TYPE:
            count = count + 1
            continue
        elif current_type == POSITIVE_TYPE:
            pos_end = read.get_closing_price_from_numeric_data(num_data[i-1:i])[0][0]
            record = np.array([pos_start, pos_end, count])
            consec_day.append(record)

            neg_start = read.get_opening_price_from_numeric_data(num_data[i:i+1])[0][0]
            count = 1
        else:
            neg_start = read.get_opening_price_from_numeric_data(num_data[i:i+1])[0][0]
            count = 1
        current_type = NEGATIVE_TYPE


#append last end
if current_type == POSITIVE_TYPE:
    pos_end = read.get_closing_price_from_numeric_data(num_data[i:i+1])[0][0]
    record = np.array([pos_start, pos_end, count])
    consec_day.append(record)
else:
    neg_end = read.get_closing_price_from_numeric_data(num_data[i:i+1])[0][0]
    record = np.array([neg_start, neg_end, count])
    consec_day.append(record)

np_consec_day = np.array(consec_day)

bar.show(np_consec_day, title)


In [7]:
data = np.zeros(shape=(4, mod))

for i in range (mod):
    data[0, i] = probability.get_positive_probability_of_day(shaped_perc[:,i])
    data[1, i], data[2, i] = absolute.get_mean_and_deviation_of_day(shaped_perc[:,i])
    data[3, i], _ = absolute.get_mean_and_deviation_of_day(shaped_range[:,i])

df = pd.DataFrame(data, columns=['Mon','Tue', 'Wed', 'Thu', 'Fri'], index=['Probability of positive','Mean','Deviation','Mean Range'])
print df


                              Mon       Tue       Wed       Thu       Fri
Probability of positive  0.636364  0.333333  0.307692  0.357143  0.538462
Mean                     2.634836 -0.103787  0.168934  0.435791  1.611207
Deviation                4.095456  2.895692  5.180865  3.637383  3.620270
Mean Range               5.420982  6.537508  7.091154  6.338926  5.462239
